Install the libraries

In [ ]:
!pip install -r requirements.txt

Install torch

In [15]:
!pip uninstall torch -y

Found existing installation: torch 2.3.0
Uninstalling torch-2.3.0:
  Successfully uninstalled torch-2.3.0


In [16]:
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.17.1%2Bcu118-cp312-cp312-win_amd64.whl (4.9 MB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0
    Uninstalling torchvision-0.18.0:
      Successfully uninstalled torchvision-0.18.0


Check GPU

In [1]:
import torch
torch.__version__


'2.2.1+cu118'

In [ ]:
torch.cuda.device_count()


In [ ]:
!nvcc --version

Check the library list

In [18]:
!pip list

Package                 Version
----------------------- --------------------
absl-py                 2.1.0
albumentations          1.4.7
annotated-types         0.7.0
asttokens               2.4.1
certifi                 2024.2.2
charset-normalizer      3.3.2
colorama                0.4.6
comm                    0.2.2
contourpy               1.2.1
cycler                  0.12.1
Cython                  3.0.10
debugpy                 1.6.7
decorator               5.1.1
exceptiongroup          1.2.0
executing               2.0.1
filelock                3.14.0
fonttools               4.52.4
fsspec                  2024.5.0
gitdb                   4.0.11
GitPython               3.1.43
grpcio                  1.64.0
idna                    3.7
imageio                 2.34.1
importlib_metadata      7.1.0
intel-openmp            2021.4.0
ipykernel               6.29.3
ipython                 8.24.0
jedi                    0.19.1
Jinja2                  3.1.4
joblib                  1.4.2
jupyt

In [ ]:
!pip install albumentations==1.4.8

need torch torchvision torchaudio with cu118

----

add `proto = proto[2]` after 
` # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred, proto = model(im, augment=augment, visualize=visualize)[:2]`

in `predict.py`


download and add gelan-c-seg.pt from [link](https://github.com/WongKinYiu/yolov9)

In [4]:
!python segment/predict.py --source test_img.jpg --img 640 --device 0 --weights gelan-c-seg.pt

segment\predict: weights=['gelan-c-seg.pt'], source=test_img.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1, retina_masks=False
YOLO  2024-5-28 Python-3.12.3 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)

Fusing layers... 
Model summary: 414 layers, 27423808 parameters, 0 gradients, 144.6 GFLOPs
image 1/1 C:\Users\yxzhh\Desktop\Machine_Learning\YOLOV9\test_img.jpg: 640x640 1 horse, 21.4ms
Speed: 0.0ms pre-process, 21.4ms inference, 1.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\predict-seg\exp20


Train customed dataset

prepare the dataset online in website like roboflow
prepare:
dataset
`data.yaml`data file
`glen-c-seg.yaml`config file
mention the location of the file below

--batch default 16, set to 8 if GPU memory small, 32 if it's big 

In [1]:
!python segment/train.py --workers 8 --device 0 --batch 8 --data data_trail_240528.yaml --img 640 --cfg models/segment/gelan-c-seg-custom-trail_240528.yaml --weights gelan-c-seg.pt --name gelan-c-seg --hyp hyp.scratch-high.yaml --no-overlap --epochs 10 --close-mosaic 10

^C


Inference with the data, 
1. change the `--source` for the image folder or image.
2. change the  `--weights` for the weights.
3. add `--save-txt` to save the output of prediction as a txt file 

In [2]:
!python segment/predict.py --source ../custom_dataset/test/images/test_4.jpg --img 640 --device 0 --weights runs/train-seg/gelan-c-seg10/weights/best.pt --save-txt

segment\predict: weights=['runs/train-seg/gelan-c-seg10/weights/best.pt'], source=../custom_dataset/test/images/test_4.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1, retina_masks=False
YOLO  2024-5-28 Python-3.12.3 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)

Fusing layers... 
gelan-c-seg-custom-trail_240528 summary: 414 layers, 27366754 parameters, 0 gradients, 144.2 GFLOPs
image 1/1 C:\Users\yxzhh\Desktop\Machine_Learning\YOLO\YOLOV9\custom_dataset\test\images\test_4.jpg: 640x384 1 bat, 1 gloves, 1 helmet, 1 pads, 1 stumps, 155.7ms
Speed: 0.0ms pre-process, 155.7ms inference, 2.0ms NMS per image at shape (1, 3, 640,